# Проект

Задание для проекта

1.	Выгрузить описания открытых вакансий с любой из площадок (hh.ru, rabota.sber.ru, indeed.com) и сохранить их в БД
2.	Предообработать текст описаний вакансий (токенизировать, нормировать)
3.	Взять любой текст, описывающий опыт из резюме, и аналогично предобработать
4.	Посчитать векторное представление описаний вакансий и опыта из резюме
5.	Посчитать косинусную меру сходства для опыта и описаний вакансий
6.	Ранжировать вакансии по мере их сходства с опытом
7.	Вывести Топ-10 наиболее релевантных
8.	Подготовить скрипт принимающий на вход текстовое описание опыта и выводящий Топ-10 наиболее релевантных вакансий из 1000 случайных


In [2]:
# !pip install ipywidgets
# !pip install pymorphy2
# !pip install pymystem3
# nltk.download('stopwords')

In [1]:
import requests
import json
import codecs
from tqdm.auto import tqdm
from collections import defaultdict
import pickle
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display, clear_output

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

from nltk.corpus import stopwords
from nltk import word_tokenize, regexp_tokenize
import nltk

import re
from string import punctuation, printable

from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, Button, Textarea, FileUpload, Layout

%matplotlib inline

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [3]:
stop_russian = stopwords.words('russian')
sber = 3529
num_per_page = 100
moscow = 1
page = 1
url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={moscow}'

In [4]:
num_pages = requests.get(url).json().get('pages')
print(f'Всего страниц с вакансиями: {num_pages}')

Всего страниц с вакансиями: 12


In [5]:
#формируем список всех id вакансий Сбера

all_vacancy_ids= []
for page in tqdm(range(num_pages)):
    url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={moscow}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    all_vacancy_ids.extend(vacancy_ids)
print(f'Всего вакансий: {len(all_vacancy_ids)}')

  0%|          | 0/12 [00:00<?, ?it/s]

Всего вакансий: 1103


In [8]:
# Извлекаем описания вакансий

df = pd.DataFrame()
columns = ['id', 'created_at', 'name', 'description', 'key_skills']
test_id = list(pd.Series(all_vacancy_ids).sample(100)) #100 случайных id для тестов
for n in tqdm(all_vacancy_ids):  #all_vacancy_ids
    url_id = f'https://api.hh.ru/vacancies/{n}'
    res = requests.get(url_id).json()
    df_ = pd.json_normalize(res)[columns]
    df = pd.concat([df, df_], axis=0)
    #dict_id_desc[n] = res_id.json().get('description')

# Переведите даты публикаций в datetime
df['created_at'] = df['created_at'].astype('datetime64[ns]').dt.strftime('%Y-%m-%d')
# df['created_date'] = df['created_at'].dt.strftime('%Y-%m-%d')
df.reset_index(drop=True)

In [9]:
# сохраним DataFrame, чтобы не скачивать каждый раз
df.to_pickle('all_raw_description.pkl', compression='zip')

# проверка что все нормально извлекается
df = pd.read_pickle('all_raw_description.pkl', compression='zip').reset_index(drop=True)
df['created_at'] = df['created_at'].astype('datetime64[ns]').dt.strftime('%Y-%m-%d')
df.head(5)

,id,created_at,name,description,key_skills
0,55415805,2022-09-07,Менеджер по работе с клиентами,"<p>Сбер — лучший работодатель России. У нас более 60 компаний экосистемы и 14 тысяч подразделений по всей стране.</p> <p>Сбер ищет клиентского менеджера для работы в мобильных офисах. Это специалист, который консультирует клиентов по продуктам, услугам и сервисам Сбера в торговых центрах, аэропортах и гипермаркетах. Эта работа для активных и общительных людей, которые стремятся к карьере и финансовому достатку.</p> <p><strong>Ты подходишь нам, если ты готов:</strong></p> <ul> <li>продавать банковские продукты и сервисы в мобильных офисах Сбера: торговых центрах, магазинах, гипермаркетах, аэропортах</li> <li>рассказывать клиентам о преимуществах сервисов Сбера и экосистемы</li> <li>помогать клиентам оформлять продукты и услуги Сбера.</li> </ul> <p>Опыт работы консультантом или промоутером приветствуется.</p> <p><strong>Работа в Сбере – это:</strong></p> <ul> <li>стабильный оклад и социальная поддержка сотрудников</li> <li>официальное оформление с первого дня</li> <li>гибкий график работы, можно совмещать с учебой</li> <li>мобильная связь и корпоративный ipad</li> <li>оплата транспортных расходов</li> <li>корпоративная пенсионная программа</li> <li>расширенный ДМС с первого дня и льготное страхование для близких</li> <li>бесплатная подписка СберПрайм+, скидки на продукты компаний-партнеров</li> <li>корпоративное обучение в Виртуальной школе Сбера </li> <li>ипотека выгоднее на 4% для каждого сотрудника.</li> </ul> <p><strong>Присоединяйся к команде клиентских менеджеров Сбера!</strong></p>",[]
1,69355530,2022-09-23,Финансовый консультант,"<p>Сбер — лучший работодатель России. У нас более 60 компаний экосистемы и 14 тысяч подразделений по всей стране.</p> <p>При оформлении карт Сбера через приложение, клиент может выбрать доставку в удобное место и время. Финансовый консультант развозит банковские продукты Сбера клиентам, а при встрече предлагает дополнительные услуги или сервисы и помогает с их оформлением. Эта работа для тех, кому интересно развиваться в банковской сфере и привлекает работа вне офиса.</p> <p><strong>Тебе предстоит:</strong></p> <ul> <li>встречаться с 7-15 клиентами в день для выдачи банковских карт по удобному для клиента адресу</li> <li>перемещаться по городу в течение дня</li> <li>продавать клиенту на встрече дополнительные продукты и услуги Сбера.</li> </ul> <p><strong>Работа в Сбере – это:</strong></p> <ul> <li>стабильный оклад и социальная поддержка сотрудников</li> <li>официальное оформление с первого дня</li> <li>возможность выбрать удобный график</li> <li>расширенный ДМС с первого дня и льготное страхование для близких</li> <li>корпоративный ipad</li> <li>оплата транспортных расходов и мобильной связи</li> <li>теплая база клиентов</li> <li>корпоративное обучение в Виртуальной школе Сбера</li> <li>бесплатная подписка СберПрайм+, скидки на продукты компаний-партнеров</li> <li>корпоративная пенсионная программа</li> <li>ипотека выгоднее на 4% для каждого сотрудника.</li> </ul> <p><strong>Присоединяйся к команде финансовых консультантов Сбера!</strong></p>",[]
2,45790891,2022-09-23,Менеджер по работе с ключевыми клиентами малого бизнеса,"<p>Сбер — лучший работодатель России. У нас более 60 компаний экосистемы и 14 тысяч подразделений по всей стране.</p> <p> </p> <p>Присоединяйся к команде малого бизнеса Сбера! На позиции «Менеджера по работе с ключевыми клиентами» ты будешь общаться с предпринимателями и руководителями компаний малого бизнеса и предлагать лучшие решения от Сбера.</p> <p>Эта вакансия для энергичных и общительных людей, готовых вникать в тонкости бизнеса наших клиентов. Мы поможем тебе освоить специальность, сформируем для тебя базу клиентов, предоставим возможности карьерного роста.</p> <p><strong>Тебе предстоит:</strong></p> <ul> <li>развивать бизнес клиентов из закрепленной базы, предлагая продукты и услуги Сбера</li> <li>проводить переговоры с клиентами на территории компаний-партнеров</li> <li

In [10]:
# функция очистки текста от html-тегов,
# в итоге оставим только русские и латинские буквы

def clean_description(row):
    return row.str.replace(r'<[^<>]*>', ' ', regex=True)\
                  .replace(r'[^А-Яа-яёЁa-zA-Z]+', ' ', regex=True)
                    
# функция лемматизации (для токенизации - nltk.regexp_tokenize) -
# длительность лемматизации примерно 45секунд
def tokenize_n_normalize(row, pat=r"\b\w\w+\b", morph=MorphAnalyzer()):
    return [morph.parse(tok)[0].normal_form 
            for tok in regexp_tokenize(row, pat)]

# очистим и лемматизируем текст описания вакансий
df["clean_desc"] = clean_description(df["description"])
df["clean_norm_desc"] = df["clean_desc"].map(lambda x: " ".join(tokenize_n_normalize(x)))
    

In [11]:
# создание виджета для загрузки текствого файла описания резюме
uploader = FileUpload(accept='.txt', multiple=False)
display(uploader)

# Создаем кнопку с нужными параметрами
button_find = Button(description="Найти релевантные резюме", 
                     button_style='success',
                     layout=Layout(width='20%', height='30px')
                        )

def on_button_clicked(b): # Описываем обработчик события нажатия на кнопку
    
    #извлекаем текст резюме из загруженного файла
    for uploaded_filename in uploader.value: 
        resume_title = uploaded_filename
        content = uploader.value[uploaded_filename]['content']   
    resume = codecs.decode(content, encoding="utf-8")  

    # очистим и лемматизируем текст описания резюме
    df_resume = pd.DataFrame([resume], columns=['text_resume'])
    df_resume['clean_text'] = clean_description(df_resume['text_resume'])
    df_resume["clean_norm_text"] = df_resume["clean_text"].map(lambda x: " ".join(tokenize_n_normalize(x)))

    # векторизация описаний вакансий и резюме
    tfidf_vec = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=10000)
    X_desc = tfidf_vec.fit_transform(df['clean_norm_desc'])
    X_resume = tfidf_vec.transform(df_resume['clean_norm_text'])

    # поиск косинусной меры сходства и 10 наиболее похожих вакансий
    cos_sim = cosine_similarity(X_resume, X_desc).flatten()
    df_cossim = pd.DataFrame(cos_sim, columns=['cos_sim'])
    index_sim10 = df_cossim.nlargest(10, ["cos_sim"]).index
    df_vacancies_for_resume = pd.merge(df, df_cossim, left_index=True, right_index=True)
    
    
    # распечатка результатов
    print(f'Вакансии для резюме: {resume_title.split(sep=".")[0]}')
    display(df_vacancies_for_resume[['id', "cos_sim", 'created_at','name']].iloc[index_sim10])

df_vacancies_for_resume = button_find.on_click(on_button_clicked) # Назначаем обработчик на событие "on_click"
display(button_find) # Отображаем кнопку

FileUpload(value={}, accept='.txt', description='Upload')

Button(button_style='success', description='Найти релевантные резюме', layout=Layout(height='30px', width='20%…

Вакансии для резюме: TEST_Менеджер по привлечению клиентов


,id,cos_sim,created_at,name
3,45381514,0.933217,2022-09-01,Менеджер по привлечению корпоративных клиентов
616,67498659,0.933217,2022-09-13,Менеджер по привлечению корпоративных клиентов
1087,67399519,0.933217,2022-09-25,Старший менеджер по привлечению корпоративных клиентов
2,45790891,0.846762,2022-09-23,Менеджер по работе с ключевыми клиентами малого бизнеса
1083,67399016,0.846762,2022-09-25,Менеджер по работе с ключевыми клиентами малого бизнеса
105,68217992,0.423099,2022-08-26,Клиентский менеджер малого и микробизнеса
31,67499784,0.314016,2022-09-20,Старший клиентский менеджер
39,67500653,0.314016,2022-09-21,Старший клиентский менеджер
45,67500055,0.314016,2022-09-23,Старший клиентский менеджер
62,67349462,0.314016,2022-09-22,Старший клиентский менеджер


Вакансии для резюме: Аналитик


,id,cos_sim,created_at,name
237,69468631,0.181275,2022-09-05,Главный экономист Управления планирования и финансового анализа
58,69664316,0.161497,2022-09-09,Аналитик данных/Разработчик отчетности
59,70194502,0.160474,2022-09-23,Data Analyst / Data Scientist в Сеть продаж
474,69820986,0.152699,2022-09-14,Эксперт (по направлению торговый эквайринг)
323,69870384,0.146026,2022-09-15,Data Analyst (Клиентская аналитика)
56,69356019,0.140004,2022-08-31,Финансовый менеджер/аналитик
269,70284746,0.139055,2022-09-24,Руководитель проектов разработки и реализации стратегии
556,69479255,0.136942,2022-09-05,Менеджер по оценке бизнеса и корпоративным финансам
254,69540885,0.135186,2022-09-06,Методолог
518,69317371,0.130737,2022-08-30,"Эксперт по направлению контроля качества, эффективности и производственной отчетности"


Вакансии для резюме: Финансовый менеджер_аналитик


,id,cos_sim,created_at,name
244,69245210,0.219379,2022-08-29,UX-исследователь (Sber CIB)
786,69592952,0.200259,2022-09-07,Руководитель направления развития систем идентификации и аутентификации
33,69651909,0.171022,2022-09-09,Юрист в команду Цифровых поверхностей Салют (реклама/коммерция)
1038,69854419,0.163452,2022-09-21,Руководитель направления в Отдел правового сопровождения международного развития
1019,69407214,0.161539,2022-09-21,Юрисконсульт в Отдел правового сопровождения международного развития Банка
182,55475765,0.160435,2022-09-01,UX/UI дизайнер
738,69034914,0.150699,2022-09-21,Customer Journey Expert (CJE)
800,69850607,0.146420,2022-09-15,M&A юрист
343,69249945,0.135549,2022-08-29,ML Researcher
917,69854731,0.133532,2022-09-21,Главный юрисконсульт (корпоративное право)
